# Analyze a large dataset with Google BigQuery

**Learning Objectives**

In this lab, you use BigQuery to:
- Access an ecommerce dataset
- Look at the dataset metadata
- Remove duplicate entries
- Write and execute queries
___

## Introduction 
BigQuery is Google's fully managed, NoOps, low cost analytics database. With BigQuery you can query terabytes and terabytes of data without having any infrastructure to manage or needing a database administrator. BigQuery uses SQL and can take advantage of the pay-as-you-go model. BigQuery allows you to focus on analyzing data to find meaningful insights.

We have a publicly available ecommerce dataset that has millions of Google Analytics records for the Google Merchandise Store loaded into a table in BigQuery. In this lab, you use a copy of that dataset. Sample scenarios are provided, from which you look at the data and ways to remove duplicate information. The lab then steps you through further analysis the data.

BigQuery can be accessed by its own browser-based interface, Google Data Studio, and many third party tools.  In this lab you will use the BigQuery Command Line interface exposed to the JuypterLab notebook via a Python library.

To follow and experiment with the BigQuery queries provided to analyze the data, see Standard SQL Query Syntax.
___

### Set up the notebook environment

__VERY IMPORTANT__:  In the cell below you must replace the text 'QWIKLABSPROJECT' with your Qwiklabs Project Name as provided during the setup of your environment. Please leave any surrounding single quotes in place.

In [9]:
PROJECT = 'QWIKLABSPROJECT' #TODO Replace with your Qwiklabs PROJECT

In [10]:
import os
os.environ["PROJECT"] = PROJECT

## Explore eCommerce data and identify duplicate records

Scenario: Your data analyst team exported the Google Analytics logs for an ecommerce website into BigQuery and created a new table of all the raw ecommerce visitor session data.

Any cell that starts with `%%bigquery` (the BigQuery Magic) will be interpreted as a SQL query that is executed on BigQuery, and the result is printed to our notebook.

BigQuery supports [two flavors](https://cloud.google.com/bigquery/docs/reference/standard-sql/migrating-from-legacy-sql#comparison_of_legacy_and_standard_sql) of SQL syntax: legacy SQL and standard SQL. The preferred is standard SQL because it complies with the official SQL:2011 standard. To instruct BigQuery to interpret our syntax as such we start the query with `#standardSQL`.

Our first query is accessing the BigQuery Information Schema which stores all object-related metadata.  In this case we want to see metadata details for the "all_sessions_raw" table. 

Tip: To run the current cell you can click the cell and hit **shift enter**

In [11]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
   * EXCEPT(table_catalog, table_schema, is_generated, generation_expression, is_stored, is_updatable,
           is_hidden, is_system_defined, is_partitioning_column, clustering_ordinal_position)
 FROM
   `data-to-insights.ecommerce.INFORMATION_SCHEMA.COLUMNS`
 WHERE
   table_name="all_sessions_raw"

,table_name,column_name,ordinal_position,is_nullable,data_type
0,all_sessions_raw,fullVisitorId,1,YES,STRING
1,all_sessions_raw,channelGrouping,2,YES,STRING
2,all_sessions_raw,time,3,YES,INT64
3,all_sessions_raw,country,4,YES,STRING
4,all_sessions_raw,city,5,YES,STRING
5,all_sessions_raw,totalTransactionRevenue,6,YES,INT64
6,all_sessions_raw,transactions,7,YES,INT64
7,all_sessions_raw,timeOnSite,8,YES,INT64
8,all_sessions_raw,pageviews,9,YES,INT64
9,all_sessions_raw,sessionQualityDim,10,YES,INT64


Let's examine how many rows are in the table.

In [12]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
    count(*)
FROM
    `data-to-insights.ecommerce.all_sessions_raw`

,f0_
0,21552195


Now, let's take a quick at few rows of data in the table.

In [13]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
    *
FROM
    `data-to-insights.ecommerce.all_sessions_raw`
limit 7

,fullVisitorId,channelGrouping,time,country,city,totalTransactionRevenue,transactions,timeOnSite,pageviews,sessionQualityDim,...,itemQuantity,itemRevenue,transactionRevenue,transactionId,pageTitle,searchKeyword,pagePathLevel1,eCommerceAction_type,eCommerceAction_step,eCommerceAction_option
0,7132224446939638973,Organic Search,0,Canada,Montreal,None,None,None,1,None,...,None,None,None,None,Apparel,None,/store.html,0,1,None
1,4356300440736175010,Paid Search,0,United States,San Francisco,None,None,None,1,None,...,None,None,None,None,Google,None,/store.html,0,1,None
2,9207254706341464048,Organic Search,0,United States,not available in demo dataset,None,None,None,1,None,...,None,None,None,None,Bags | Google Merchandise Store,None,/google+redesign/,0,1,None
3,5050081640297625286,Organic Search,0,United States,not available in demo dataset,None,None,None,1,None,...,None,None,None,None,Bags | Google Merchandise Store,None,/google+redesign/,0,1,None
4,8914092422212122129,Direct,0,Canada,not available in demo dataset,None,None,None,1,None,...,None,None,None,None,Bags,None,/google+redesign/,0,1,None
5,8424572407219339151,Direct,0,India,Hyderabad,None,None,None,1,None,...,None,None,None,None,Bags,None,/google+redesign/,0,1,None
6,7790889878428767130,Organic Search,0,Puerto Rico,not available in demo dataset,None,None,None,1,None,...,None,None,None,None,Office | Google Merchandise Store,None,/google+redesign/,0,1,None


### Identify duplicate rows

Seeing a sample amount of data may give you greater intuition for what is included in the dataset. But since the table is quite large, a preview is not likely to render meaningful resutls.  As you scan and scroll through the sample rows you see there is no singular field that uniquely identifies a row, so you need advanced logic to identify duplicate rows.

The query below uses the SQL GROUP BY function on every field and counts (COUNT) where there are rows that have the same values across every field.

If every field is unique, the COUNT will return 1 as there are no other groupings of rows with the exact same value for all fields.
If there is a row with the same values for all fields, they will be grouped together and the COUNT will be greater than 1. The last part of the query is an aggregation filter using HAVING to only show the results that have a COUNT of duplicates greater than 1.
Run the following query to find duplicate records across all columns.

In [14]:
%%bigquery --project $PROJECT
#standardSQL
SELECT COUNT(*) as num_duplicate_rows, * FROM
`data-to-insights.ecommerce.all_sessions_raw`
GROUP BY
fullVisitorId, channelGrouping, time, country, city, totalTransactionRevenue, transactions, 
timeOnSite, pageviews, sessionQualityDim, date, visitId, type, productRefundAmount, productQuantity, 
productPrice, productRevenue, productSKU, v2ProductName, v2ProductCategory, productVariant, 
currencyCode, itemQuantity, itemRevenue, transactionRevenue, transactionId, pageTitle, 
searchKeyword, pagePathLevel1, eCommerceAction_type, eCommerceAction_step, eCommerceAction_option
HAVING num_duplicate_rows > 1;

,num_duplicate_rows,fullVisitorId,channelGrouping,time,country,city,totalTransactionRevenue,transactions,timeOnSite,pageviews,...,itemQuantity,itemRevenue,transactionRevenue,transactionId,pageTitle,searchKeyword,pagePathLevel1,eCommerceAction_type,eCommerceAction_step,eCommerceAction_option
0,14,4191489896925210475,Referral,814159,United States,Kirkland,351000000.0,1.0,1500.0,67,...,None,None,351000000.0,ORD201612212603,Checkout Confirmation,None,/ordercompleted.html,6,1,None
1,2,7593755608800876778,Organic Search,1755066,Philippines,not available in demo dataset,NaN,NaN,1856.0,25,...,None,None,NaN,None,Men's Apparel | Google Merchandise Store,None,/google+redesign/,2,1,None
2,2,4608742628260942878,Direct,42929,United States,San Jose,NaN,NaN,191.0,20,...,None,None,NaN,None,Office,None,/google+redesign/,1,1,None
3,2,4596351988415872717,Direct,170,United States,Palo Alto,NaN,NaN,114.0,13,...,None,None,NaN,None,Nest-USA,None,/google+redesign/,2,1,None
4,2,2190964662485966036,Direct,334367,Japan,Osaka,NaN,NaN,380.0,27,...,None,None,NaN,None,Kids' Apparel | Google Merchandise Store,None,/google+redesign/,2,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,6,7360768108223062056,Display,2222009,United States,Sunnyvale,NaN,NaN,2262.0,16,...,None,None,NaN,None,Payment Method,None,/payment.html,5,2,Payment
611,6,043584487743669327,Referral,318814,United States,Austin,192560000.0,2.0,346.0,21,...,None,None,151000000.0,ORD201706192356,Checkout Confirmation,None,/ordercompleted.html,6,1,None
612,2,6891422330853688882,Referral,56,United States,San Francisco,274000000.0,1.0,554.0,16,...,None,None,NaN,None,Nest-USA,None,/google+redesign/,2,1,None
613,2,552850508976566436,Organic Search,67845,Italy,not available in demo dataset,NaN,NaN,97.0,23,...,None,None,NaN,None,Headgear | Apparel | Google Merchandise Store,None,/store.html,1,1,None


As you can see there are quite a few "duplicate" records (615) when analyzed with these parameters.

In your own datasets, even if you have a unique key, it is still beneficial to confirm the uniqueness of the rows with COUNT, GROUP BY, and HAVING before you begin your analysis.

## Analyze the new all_sessions table

In this section you use a deduplicated table called all_sessions.

Scenario: Your data analyst team has provided you with a relevant query, and your schema experts have identified the key fields that must be unique for each record per your schema.

Run the query to confirm that no duplicates exist, this time against the "all_sessions" table:

In [15]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
fullVisitorId, # the unique visitor ID
visitId, # a visitor can have multiple visits
date, # session date stored as string YYYYMMDD
time, # time of the individual site hit  (can be 0 to many per visitor session)
v2ProductName, # not unique since a product can have variants like Color
productSKU, # unique for each product
type, # a visitor can visit Pages and/or can trigger Events (even at the same time)
eCommerceAction_type, # maps to ‘add to cart', ‘completed checkout'
eCommerceAction_step,
eCommerceAction_option,
  transactionRevenue, # revenue of the order
  transactionId, # unique identifier for revenue bearing transaction
COUNT(*) as row_count
FROM
`data-to-insights.ecommerce.all_sessions`
GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12
HAVING row_count > 1 # find duplicates

,fullVisitorId,visitId,date,time,v2ProductName,productSKU,type,eCommerceAction_type,eCommerceAction_step,eCommerceAction_option,transactionRevenue,transactionId,row_count


The query returns zero records.

## Write basic SQL against the eCommerce data

In this section, you query for insights on the ecommerce dataset.

A good first path of analysis is to find the total unique visitors
The query below determines the total views by counting product_views and the number of unique visitors by counting fullVisitorID.

In [16]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
  COUNT(*) AS product_views,
  COUNT(DISTINCT fullVisitorId) AS unique_visitors
FROM `data-to-insights.ecommerce.all_sessions`;

,product_views,unique_visitors
0,21493109,389934


The next query shows total unique visitors(fullVisitorID) by the referring site (channelGrouping):

In [17]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
  COUNT(DISTINCT fullVisitorId) AS unique_visitors,
  channelGrouping
FROM `data-to-insights.ecommerce.all_sessions`
GROUP BY 2
ORDER BY 2 DESC;

,unique_visitors,channelGrouping
0,38101,Social
1,57308,Referral
2,11865,Paid Search
3,211993,Organic Search
4,3067,Display
5,75688,Direct
6,5966,Affiliates
7,62,(Other)


To find deeper insights in the data, the next query lists the five products with the most views (product_views) from unique visitors. The query counts number of times a product (v2ProductName) was viewed (product_views), puts the list in descending order, and lists the top 5 entries:

In [18]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
  COUNT(*) AS product_views,
  (v2ProductName) AS ProductName
FROM `data-to-insights.ecommerce.all_sessions`
WHERE type = 'PAGE'
GROUP BY v2ProductName
ORDER BY product_views DESC
LIMIT 5;

,product_views,ProductName
0,316482,Google Men's 100% Cotton Short Sleeve Hero Tee...
1,221558,22 oz YouTube Bottle Infuser
2,210700,YouTube Men's Short Sleeve Hero Tee Black
3,202205,Google Men's 100% Cotton Short Sleeve Hero Tee...
4,200789,YouTube Custom Decals


Now expand your previous query to include the total number of distinct products ordered and the total number of total units ordered (productQuantity):

In [19]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
  COUNT(*) AS product_views,
  COUNT(productQuantity) AS orders,
  SUM(productQuantity) AS quantity_product_ordered,
  v2ProductName
FROM `data-to-insights.ecommerce.all_sessions`
WHERE type = 'PAGE'
GROUP BY v2ProductName
ORDER BY product_views DESC
LIMIT 5;

,product_views,orders,quantity_product_ordered,v2ProductName
0,316482,3158,6352,Google Men's 100% Cotton Short Sleeve Hero Tee...
1,221558,508,4769,22 oz YouTube Bottle Infuser
2,210700,949,1114,YouTube Men's Short Sleeve Hero Tee Black
3,202205,2713,8072,Google Men's 100% Cotton Short Sleeve Hero Tee...
4,200789,1703,11336,YouTube Custom Decals


Lastly, lets' expand the query to include the average amount of product per order (total number of units ordered/total number of orders, or SUM(productQuantity)/COUNT(productQuantity)).

In [20]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
  COUNT(*) AS product_views,
  COUNT(productQuantity) AS orders,
  SUM(productQuantity) AS quantity_product_ordered,
  SUM(productQuantity) / COUNT(productQuantity) AS avg_per_order,
  (v2ProductName) AS ProductName
FROM `data-to-insights.ecommerce.all_sessions`
WHERE type = 'PAGE'
GROUP BY v2ProductName
ORDER BY product_views DESC
LIMIT 5;

,product_views,orders,quantity_product_ordered,avg_per_order,ProductName
0,316482,3158,6352,2.011400,Google Men's 100% Cotton Short Sleeve Hero Tee...
1,221558,508,4769,9.387795,22 oz YouTube Bottle Infuser
2,210700,949,1114,1.173867,YouTube Men's Short Sleeve Hero Tee Black
3,202205,2713,8072,2.975304,Google Men's 100% Cotton Short Sleeve Hero Tee...
4,200789,1703,11336,6.656489,YouTube Custom Decals


We see that among these top 5 products by product views that the 22 oz YouTube Bottle Infuser had the highest avg_per_order with 9.38 units per order.

Copyright 2019 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.